In [1]:
import numpy as np
import math
import gymnasium as gym
from gymnasium import spaces
import copy
from enum import Enum, auto

In [2]:
np.set_printoptions(precision=3, suppress=True)

In [3]:
np.random.seed(0)

In [ ]:
class FunctionType(Enum):
    SIN = auto()
    ARCSIN = auto()
    LOG = auto()
    EXP = auto()

    @property
    def inverse(self):
        """Simple inverse mapping."""
        return {
            FunctionType.SIN: FunctionType.ARCSIN,
            FunctionType.ARCSIN: FunctionType.SIN,
            FunctionType.LOG: FunctionType.EXP,
            FunctionType.EXP: FunctionType.LOG,
        }[self]

In [ ]:
class Func:
    """Represents f(x) = op(sub(x))."""
    def __init__(self, ftype: FunctionType, sub=None):
        self.type = ftype
        self.sub = sub
        self.complexity = 1 if sub is None else sub.complexity + 1

    def apply(self, ftype):
        if ftype == self.type.inverse:
            return self.sub
        return Func(ftype, self)

    def evaluate(self, x):
        """Safe evaluation (no NaNs)."""
        if self.sub:
            x = self.sub.evaluate(x)

        try:
            if self.type == FunctionType.SIN:
                return math.sin(x)
            if self.type == FunctionType.ARCSIN:
                x = max(-0.999999, min(0.999999, x))
                return math.asin(x)
            if self.type == FunctionType.LOG:
                return math.log(abs(x) + 1e-6)
            if self.type == FunctionType.EXP:
                return math.exp(max(-50, min(50, x)))

        except:
            return 0.0

    def __repr__(self):
        return f"{self.type.name}({self.sub})"

In [ ]:
class FunctionSimplifyEnv(gym.Env):
    """
    Observation: (x, f(x)) sampled over range.
    Action: choose function to apply.
    Goal: reduce composition depth to 0.
    """

    metadata = {"render.modes": ["human"]}

    def __init__(self, initial_func: Func, n_samples=500, allow_arguments=False):
        super().__init__()

        # Discrete set of possible actions
        self.action_space = spaces.Discrete(len(FunctionType))

        # Observations = (x, y)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(n_samples, 2), dtype=np.float32
        )

        self.initial_func = initial_func
        self.current_func = copy.deepcopy(initial_func)
        self.x = np.linspace(-5, 5, n_samples)
        self.allow_arguments = allow_arguments  # ← optional extension

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_func = copy.deepcopy(self.initial_func)
        return self._get_obs(), {}

    def _get_obs(self):
        if self.current_func is None:
            return np.stack([self.x, self.x], axis=-1).astype(np.float32)
        y = np.array([self.current_func.evaluate(v) for v in self.x])
        y = np.nan_to_num(y, 0.0)
        return np.stack([self.x, y], axis=-1).astype(np.float32)

    def step(self, action_idx, step_count):
        ftype = list(FunctionType)[action_idx]

        prev_complexity = self.current_func.complexity
        self.current_func = self.current_func.apply(ftype)
        if self.current_func is None:
            new_complexity = 0
            reward = 1000 * (500/(step_count+1))
            return self._get_obs(), 100, True, False, {None, 0}

        new_complexity = self.current_func.complexity

        reward = float(prev_complexity - new_complexity)
        reward -= step_count * 0.05
        if new_complexity >= 10:
            reward -= 100
            self.current_func = copy.deepcopy(self.initial_func)

        done = new_complexity == 0
        obs = self._get_obs()

        return obs, reward, done, False, {
            "func": repr(self.current_func),
            "complexity": new_complexity
        }

    def render(self):
        print("Function:", self.current_func)
        print("Complexity:", self.current_func.complexity)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ============================================================
#  DQN Network
# ============================================================
class DQN(nn.Module):
    def __init__(self, n_obs, n_actions):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_obs, 512),
            nn.PReLU(),
            nn.Linear(512, 512),
            nn.PReLU(),
            nn.Linear(512, 512),
            nn.PReLU(),
            nn.Linear(512, 512),
            nn.PReLU(),
            nn.Linear(512, n_actions)
        )

    def forward(self, x):
        return self.net(x.flatten(start_dim=1))


# ============================================================
#  Replay Buffer
# ============================================================
class ReplayBuffer:
    def __init__(self, capacity=50000):
        self.buf = deque(maxlen=capacity)

    def push(self, s, a, r, ns, d):
        self.buf.append((s, a, r, ns, d))

    def sample(self, batch_size):
        batch = random.sample(self.buf, batch_size)
        s, a, r, ns, d = map(np.array, zip(*batch))
        return (
            torch.tensor(s, dtype=torch.float32, device=device),
            torch.tensor(a, dtype=torch.int64, device=device),
            torch.tensor(r, dtype=torch.float32, device=device),
            torch.tensor(ns, dtype=torch.float32, device=device),
            torch.tensor(d, dtype=torch.float32, device=device)
        )

    def __len__(self):
        return len(self.buf)


# ============================================================
#  Epsilon-greedy action selection
# ============================================================
def select_action(policy_net, state, epsilon, n_actions):
    if random.random() < epsilon:
        return random.randrange(n_actions)
    qvals = policy_net(state.unsqueeze(0))
    return int(qvals.argmax().item())


In [ ]:
def random_function(max_depth=4):
    funcs = [FunctionType.SIN, FunctionType.ARCSIN, FunctionType.LOG, FunctionType.EXP]

    #funcs = [FunctionType.ARCSIN, FunctionType.EXP]

    f = None

    depth = max_depth#random.randint(1, max_depth)
    prev_func = None
    for _ in range(depth):
        available_funcs = [func for func in funcs if prev_func is None or func != prev_func.inverse]
        selected_func = random.choice(available_funcs)
        f = Func(selected_func, f)
        prev_func = selected_func
    print(f)

    return f

In [ ]:
r_func = random_function(2)

LOG(ARCSIN(None))


In [ ]:
r_func

LOG(ARCSIN(None))

In [ ]:
x = np.linspace(-5, 5, 500)
y = np.array([r_func.evaluate(v) for v in x])
y = np.nan_to_num(y, 0.0)
np.stack([x, y], axis=-1).astype(np.float32)

array([[-5.   ,  0.451],
       [-4.98 ,  0.451],
       [-4.96 ,  0.451],
       [-4.94 ,  0.451],
       [-4.92 ,  0.451],
       [-4.9  ,  0.451],
       [-4.88 ,  0.451],
       [-4.86 ,  0.451],
       [-4.84 ,  0.451],
       [-4.82 ,  0.451],
       [-4.8  ,  0.451],
       [-4.78 ,  0.451],
       [-4.76 ,  0.451],
       [-4.739,  0.451],
       [-4.719,  0.451],
       [-4.699,  0.451],
       [-4.679,  0.451],
       [-4.659,  0.451],
       [-4.639,  0.451],
       [-4.619,  0.451],
       [-4.599,  0.451],
       [-4.579,  0.451],
       [-4.559,  0.451],
       [-4.539,  0.451],
       [-4.519,  0.451],
       [-4.499,  0.451],
       [-4.479,  0.451],
       [-4.459,  0.451],
       [-4.439,  0.451],
       [-4.419,  0.451],
       [-4.399,  0.451],
       [-4.379,  0.451],
       [-4.359,  0.451],
       [-4.339,  0.451],
       [-4.319,  0.451],
       [-4.299,  0.451],
       [-4.279,  0.451],
       [-4.259,  0.451],
       [-4.238,  0.451],
       [-4.218,  0.451],


In [ ]:
from tqdm.notebook import tqdm

In [ ]:
import matplotlib.pyplot as plt
plt.ion()

def plot_results(steps):
  x_vals = list(range(len(steps)))
  plt.plot(x_vals, steps)
  plt.xlabel('Episode')
  plt.ylabel('Steps to completion')

In [ ]:
class Curriculum:
    def __init__(self, max_depth=6):
        self.level = 1
        self.max_depth = max_depth
        self.env = FunctionSimplifyEnv(random_function(self.level))

    def next_env(self):
        f = random_function(self.level)
        return FunctionSimplifyEnv(f)

    def step(self):
        if self.level < self.max_depth:
            self.level += 1


In [ ]:
curr = Curriculum(max_depth=6)

EXP(None)


In [ ]:
obs, _ = curr.env.reset()
n_obs = obs.size
n_actions = curr.env.action_space.n

policy_net = DQN(n_obs, n_actions).to(device)
target_net = DQN(n_obs, n_actions).to(device)

In [ ]:
def train_dqn(
    env,
    checkpoint_path="checkpoint.pth",  # Path to the checkpoint
    episodes=300,
    batch_size=64,
    gamma=0.999,
    lr=0.001,
    epsilon_start=1.0,
    epsilon_end=0.1,
    epsilon_decay=300
):
    optimizer = optim.AdamW(policy_net.parameters(), lr=lr)

    start_episode = 0

    replay = ReplayBuffer()

    epsilon = epsilon_start
    eps_decay_rate = (epsilon_start - epsilon_end) / epsilon_decay

    returns = []
    steps_to_completion = []

    for ep in tqdm(range(start_episode, episodes)):
        state, _ = env.reset()
        state = torch.tensor(state.flatten(), dtype=torch.float32, device=device)

        total_reward = 0

        action_set = [env.current_func]
        for t in range(200):
            # --- Select action ---
            action = select_action(policy_net, state, epsilon, n_actions)

            # --- Step ---
            next_state, reward, done, _, _ = env.step(action, t)
            action_set.append(env.current_func)

            if done:
                break

            next_state = torch.tensor(next_state.flatten(), dtype=torch.float32, device=device)

            total_reward += reward

            # Store in replay
            replay.push(
                state.cpu().numpy(),
                action,
                reward,
                next_state.cpu().numpy(),
                float(done)
            )

            state = next_state

            # --- Learning step ---
            if len(replay) > batch_size:
                s, a, r, ns, d = replay.sample(batch_size)

                q_curr = policy_net(s)
                q_curr = q_curr.gather(1, a.unsqueeze(1)).squeeze()

                q_next = target_net(ns).max(1).values.detach()
                q_target = r + gamma * q_next * (1 - d)

                loss = nn.SmoothL1Loss()(q_curr, q_target)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            # update target network slowly
            for tp, pp in zip(target_net.parameters(), policy_net.parameters()):
                tp.data.copy_(0.995 * tp.data + 0.005 * pp.data)

        epsilon = max(epsilon_end, epsilon - eps_decay_rate)
        returns.append(total_reward)
        steps_to_completion.append(t)

        if ep % 20 == 0 and ep > 0:
            print(steps_to_completion[-20:])

        plot_results(steps_to_completion)

    return policy_net, returns


In [ ]:
for stage in range(6):
    plt.close('all')
    for i in range(5):
      env = curr.next_env()
      print("Training depth", curr.level)
      train_dqn(env, episodes=500)
    curr.step()


EXP(None)
Training depth 1


  0%|          | 0/500 [00:00<?, ?it/s]

[110, 19, 40, 12, 19, 53, 0, 19, 0, 0, 49, 0, 9, 11, 48, 28, 0, 8, 0, 0]
[36, 34, 0, 17, 0, 47, 60, 0, 65, 24, 8, 199, 32, 83, 21, 0, 0, 26, 73, 45]
[22, 0, 2, 199, 22, 0, 93, 73, 17, 21, 30, 76, 13, 11, 111, 0, 43, 30, 0, 17]
[13, 75, 0, 90, 37, 71, 78, 91, 0, 0, 43, 0, 179, 0, 23, 2, 80, 26, 47, 0]
[17, 32, 0, 0, 26, 13, 132, 11, 0, 4, 39, 43, 30, 26, 89, 2, 55, 0, 80, 39]
[32, 129, 34, 17, 9, 199, 183, 48, 31, 187, 30, 6, 122, 15, 0, 170, 61, 30, 50, 91]
[13, 41, 0, 0, 46, 199, 9, 0, 74, 134, 37, 0, 9, 169, 100, 37, 190, 54, 44, 68]
[47, 26, 0, 11, 0, 98, 96, 83, 13, 96, 44, 164, 40, 58, 63, 2, 199, 0, 199, 131]
[11, 155, 199, 13, 183, 52, 9, 31, 21, 85, 107, 24, 2, 64, 35, 135, 13, 0, 118, 40]
[51, 63, 29, 0, 24, 82, 86, 29, 199, 57, 108, 152, 31, 42, 39, 20, 199, 87, 15, 45]
[199, 22, 193, 132, 98, 75, 0, 75, 137, 146, 46, 19, 40, 199, 38, 22, 11, 51, 0, 92]
[44, 199, 35, 24, 0, 169, 199, 0, 199, 17, 51, 52, 144, 189, 26, 72, 55, 64, 199, 68]
[78, 199, 33, 199, 173, 199, 0, 123, 5

  0%|          | 0/500 [00:00<?, ?it/s]

[60, 15, 17, 128, 100, 24, 0, 0, 85, 72, 42, 169, 0, 199, 11, 13, 162, 0, 52, 34]
[96, 64, 11, 24, 102, 36, 58, 19, 28, 96, 66, 2, 6, 87, 13, 0, 0, 41, 0, 13]
[119, 110, 81, 4, 0, 11, 102, 34, 0, 0, 47, 51, 105, 15, 35, 18, 73, 0, 0, 43]
[18, 17, 53, 74, 84, 169, 199, 0, 20, 87, 0, 199, 15, 46, 54, 13, 11, 13, 9, 0]
[103, 115, 54, 115, 122, 24, 0, 105, 199, 0, 0, 51, 162, 13, 105, 13, 2, 0, 42, 0]
[9, 13, 15, 45, 26, 13, 9, 0, 71, 90, 98, 0, 67, 19, 80, 168, 38, 137, 135, 9]
[26, 21, 0, 45, 105, 73, 0, 72, 9, 96, 39, 60, 199, 0, 199, 6, 68, 91, 42, 104]
[81, 116, 70, 47, 88, 13, 41, 49, 28, 20, 57, 43, 11, 22, 39, 26, 22, 0, 30, 17]
[0, 41, 33, 19, 35, 199, 31, 113, 87, 199, 11, 39, 81, 40, 55, 100, 11, 118, 168, 116]
[20, 26, 22, 0, 9, 26, 199, 199, 65, 185, 13, 9, 0, 199, 199, 11, 0, 159, 11, 89]
[28, 172, 55, 51, 0, 9, 154, 22, 126, 55, 112, 11, 199, 18, 199, 0, 70, 0, 0, 101]
[135, 199, 199, 2, 4, 24, 167, 112, 113, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[11, 0, 0, 0, 0, 0, 0, 0, 9, 0, 

...

[180, 199, 199, 199, 199, 136, 72, 199, 2, 186, 161, 111, 157, 199, 175, 4, 199, 199, 186, 100]
[81, 44, 165, 185, 186, 60, 191, 199, 199, 38, 4, 84, 199, 138, 39, 188, 53, 65, 12, 91]
[199, 199, 88, 199, 14, 199, 199, 147, 19, 174, 49, 130, 40, 199, 104, 6, 72, 11, 199, 90]
[199, 111, 107, 84, 199, 130, 48, 199, 159, 19, 70, 53, 72, 4, 40, 141, 13, 112, 158, 30]
[87, 23, 2, 2, 105, 169, 2, 39, 36, 179, 6, 32, 69, 2, 13, 126, 11, 4, 163, 15]
[60, 142, 36, 117, 81, 199, 28, 68, 71, 19, 4, 29, 4, 47, 13, 93, 8, 19, 45, 13]
[44, 40, 138, 4, 24, 15, 24, 37, 26, 11, 67, 27, 15, 2, 9, 15, 2, 10, 21, 11]
[4, 70, 11, 8, 50, 4, 71, 71, 4, 90, 13, 6, 2, 27, 33, 56, 33, 4, 4, 37]
[47, 11, 24, 29, 65, 9, 2, 15, 2, 26, 11, 2, 13, 2, 2, 13, 4, 15, 2, 2]
[2, 15, 2, 42, 4, 2, 4, 17, 42, 8, 2, 15, 15, 2, 9, 13, 35, 13, 24, 2]
[44, 37, 4, 9, 4, 26, 80, 2, 24, 28, 33, 21, 17, 2, 17, 2, 15, 4, 2, 2]
[39, 2, 2, 9, 21, 2, 8, 2, 2, 13, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2]
[17, 22, 2, 2, 2, 11, 26, 6, 2, 2, 2, 15, 6

  0%|          | 0/500 [00:00<?, ?it/s]

[153, 94, 199, 168, 134, 149, 27, 104, 196, 168, 93, 142, 199, 15, 199, 180, 69, 33, 104, 199]
[199, 199, 199, 199, 163, 15, 66, 111, 199, 199, 17, 199, 23, 80, 171, 199, 84, 170, 82, 199]
[199, 72, 199, 199, 132, 19, 173, 199, 199, 115, 21, 199, 199, 183, 73, 199, 138, 199, 103, 35]
[68, 19, 138, 199, 126, 199, 199, 2, 199, 199, 192, 68, 120, 28, 199, 199, 199, 199, 199, 151]
[199, 56, 199, 199, 4, 4, 199, 144, 199, 199, 199, 184, 199, 199, 32, 199, 199, 199, 199, 2]
[199, 199, 199, 21, 162, 199, 199, 165, 199, 199, 199, 199, 46, 109, 199, 187, 199, 199, 199, 199]
[199, 199, 199, 185, 199, 192, 199, 67, 199, 164, 199, 4, 163, 199, 28, 199, 199, 129, 199, 199]
[108, 199, 64, 199, 199, 199, 199, 199, 199, 199, 199, 199, 167, 186, 199, 199, 199, 199, 199, 199]
[199, 199, 199, 199, 199, 199, 199, 199, 169, 199, 199, 199, 199, 199, 199, 199, 71, 63, 199, 199]



KeyboardInterrupt

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Error in callback <function _draw_all_if_interactive at 0x7da4a161ce00> (for post_execute):
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/events.py", line 89, in trigger
    func(*args, **kwargs)
  File "/usr/local/lib/python3.12/dist-packages/matplotlib/pyplot.py", line 279, in _draw_all_if_interactive
    draw_all()
  File "/usr/local/lib/python3.12/dist-packages/matplotlib/_pylab_helpers.py", line 131, in draw_all
    manager.canvas.draw_idle()
  File "/usr/local/lib/python3.12/dist-packages/matplotlib/backend_bases.py", line 1891, in draw_idle
    self.draw(*args, **kwargs)
  File "/usr/local/lib/python3.12/dist-packages/matplotlib/backends/backend_agg.py", line 382, in draw
    self.figure.draw(self.renderer)
  File "/usr/local/lib/python3.12/dist-packages/matplotlib/artist.py", line 94, in draw_wrapper
    result = draw(artist, renderer, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/p